In order to enable Annoatation based configuration, we need to specify the intent in either XML or in Java (if we want to use Java configuration).

```xml
<beans>
    <context:annotation-config>
    <context:component-scan base-package="com.base.package" />
```

The first one activates annotations in already defined beans, second one activates annotations as well as specifies where to find annotations. Refer to [this](https://stackoverflow.com/questions/7414794/difference-between-contextannotation-config-and-contextcomponent-scan) to get a better understanding.

Equivalently,
```java
@ComponentScan(basePackages = "com.base.package")
@Configuration
public Configurations{
}
```

### Autowiring Mode
The `@Autowired` annotation can be applied to
- constructor
- method
- parameters
- fields

Once we annotate any of the above, they are autowired by type. Spring searches (where?) for the dependency by type. If more than one dependencies of the same type are available, then we get an error.
Spring searches for autowired dependency in packages listed in `component-scan` (any class annotated with `@Component`, etc). If no matching bean is found, it searches in configuration java file (file annotated with `@Configuration`) for `@Bean`. For example:  

Player.java
```java
public class Player {
	@Autowired
	private MusicFinderService musicFinderService;
	
	public void printLyrics(String song) {
		String lyrics = musicFinderService.getLyrics(song);
		System.out.println(lyrics);
	}
}
```
MusicFinderService.java:
```java
@Component
public class MusicFinderService {	
	public MusicFinderService() {
		this.type = type;
	}
	
	public String getLyrics(String song) {
		// implementation
	}
}
```

Or we can use @Bean.  
```java
@Configuration
public class ApplicationConfiguration {
	@Bean
	public MusicFinderService getMusicFinderService() {
		return new MusicFinderService();
	}
}
```

If we have multiple beans of same type, we can resolve conflict using `@Primary` and `@Qualifier`.  
```java
@Configuration
public class ApplicationConfiguration {
	@Bean
    @Qualifier("myService")
    // Or in one line @Bean("myService")    
	public MusicFinderService getMusicFinderService() {
		return new MusicFinderService();
	}
}
```
```java
public class Player {
	@Autowired
	@Qualifier("myService")
	private MusicFinderService musicFinderService;
	
	public void printLyrics(String song) {
		String lyrics = musicFinderService.getLyrics(song);
		System.out.println(lyrics);
	}
}
```
Or in case of @Component,
```java
@Component("myService")
public class MusicFinderService {	
	public MusicFinderService() {
		this.type = type;
	}
	
	public String getLyrics(String song) {
		// implementation
	}
}
```
As we can see, @Qualifier acts a bit like autowiring by name. @Primary is equivalent to setting primary attribute as true in case of XML based configuration.
```java
@Configuration
public class ApplicationConfiguration {
	@Bean
    @Primary
	public MusicFinderService getFirstMusicFinderService() {
		// implementation
	}
    
    @Bean
	public MusicFinderService getSecondMusicFinderService() {
		// implementation
	}
}
```

### @Resource
@Resource means get a known resource(bean) by name. The name is extracted from the name of the annotated setter or field, or it is taken from the name-parameter. @Resource has a built-in fallback, which kicks in when resolution by-name fails. In this case, it falls back to the @Autowired-kind resolution by-type.  

In a nutshell,

@Autowired and @Inject
- Matches by Type
- Restricts by Qualifiers
- Matches by Name

@Resource
- Matches by Name
- Matches by Type
- Restricts by Qualifiers (ignored if match is found by name)

### @Value
Typically used to inject externalized properties.

```java
@Component
public class Player {
	@Value("${player.name}")
	private String playerName;
    
    // More implementations
}
```

We can either pass in a constant literal value to the annotation or use `${}` to specify property from a properties file. To specify property file, in Java based configuration, do:

```java
@Configuration
@PropertySource("classpath:application.properties")
public class ApplicationConfiguration {
    // Implementation
}
```

In XML based configuration,
```xml
<context:property-placeholder location="classpath:application.properties"/>
```

### @Scope
Equivalent to `scope` attribute of bean tag.